# Section V.E. Model Benchmarking Results

This notebook benchmarks ARIMA, Prophet, LSTM, Random Forest, and XGBoost for time series forecasting of the simulated workload metrics in `SimulatedQueryMetrics.csv`.

> **Note:** For brevity and reproducibility, this example uses simple versions and default parameters. For robust benchmarks, perform hyperparameter tuning and use a robust cross-validation scheme.

## 1. Imports and Data Loading

In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet

# For LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('SimulatedQueryMetrics.csv', parse_dates=['MetricDate'])

## 2. Prepare Data Helper Functions

In [4]:
def get_series(df, query, variant, metric):
    s = df[(df['QueryName']==query) & (df['QueryVariant']==variant)].sort_values('MetricDate')[['MetricDate', metric]].copy()
    s = s.rename(columns={'MetricDate':'ds', metric:'y'})
    s['y'] = s['y'].interpolate().fillna(method='bfill')
    return s

# For supervised learning (tabular) models
def make_supervised(series, n_lags=5):
    df = pd.DataFrame(series)
    for i in range(1, n_lags+1):
        df[f'lag_{i}'] = df['y'].shift(i)
    df = df.dropna().reset_index(drop=True)
    X = df[[f'lag_{i}' for i in range(1, n_lags+1)]].values
    y = df['y'].values
    return X, y


## 3. Benchmarking Loop (One Query and Variant Example)
You can expand this for all queries/variants and metrics in a full run.

In [6]:
results = []
metrics = ['CPU', 'LatencyMs', 'LogicalReads']
queries = ['Q1', 'Q2']
variant = 1  # Example: use variant 1 for brevity
n_test = 48  # Last 2 days as test

for query in queries:
    for metric in metrics:
        s = get_series(df, query, variant, metric)
        train, test = s.iloc[:-n_test], s.iloc[-n_test:]

        # ARIMA
        try:
            arima = ARIMA(train['y'], order=(2,1,2)).fit()
            pred_arima = arima.forecast(steps=n_test)
            rmse_arima = np.sqrt(mean_squared_error(test['y'], pred_arima))
        except:
            rmse_arima = np.nan

        # Prophet
        try:
            m = Prophet()
            m.fit(train)
            forecast = m.predict(test[['ds']])
            pred_prophet = forecast['yhat'].values
            rmse_prophet = np.sqrt(mean_squared_error(test['y'], pred_prophet))
        except:
            rmse_prophet = np.nan

        # LSTM (robust, corrected version)
        try:
            scaler = StandardScaler()
            all_data = np.concatenate([train['y'].values, test['y'].values])
            all_scaled = scaler.fit_transform(all_data.reshape(-1, 1)).flatten()
            train_scaled = all_scaled[:len(train)]
            test_scaled = all_scaled[len(train)-5:]  # Include last 5 train for windowing

            # Create lagged sequences
            def create_lstm_data(series, n_lags=5):
                X, y = [], []
                for i in range(n_lags, len(series)):
                    X.append(series[i-n_lags:i])
                    y.append(series[i])
                X = np.array(X)
                y = np.array(y)
                return X[..., np.newaxis], y

            X_train, y_train = create_lstm_data(train_scaled, n_lags=5)
            X_test, y_test = create_lstm_data(test_scaled, n_lags=5)

            model = Sequential()
            model.add(LSTM(16, input_shape=(X_train.shape[1], X_train.shape[2])))
            model.add(Dense(1))
            model.compile(loss='mse', optimizer='adam')
            model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0)

            pred_lstm_scaled = model.predict(X_test)
            # Inverse transform
            pred_lstm = scaler.inverse_transform(pred_lstm_scaled)
            y_test_inv = scaler.inverse_transform(y_test.reshape(-1,1))
            rmse_lstm = np.sqrt(mean_squared_error(y_test_inv, pred_lstm))
        except Exception as e:
            print(f'LSTM error for {query}-{metric}:', e)
            rmse_lstm = np.nan

        # Random Forest
        try:
            X_train, y_train = make_supervised(train['y'], n_lags=5)
            X_test, y_test = make_supervised(pd.concat([train['y'][-5:], test['y']]), n_lags=5)
            rf = RandomForestRegressor(n_estimators=50)
            rf.fit(X_train, y_train)
            pred_rf = rf.predict(X_test)
            rmse_rf = np.sqrt(mean_squared_error(y_test, pred_rf))
        except:
            rmse_rf = np.nan

        # XGBoost
        try:
            xgb = XGBRegressor(n_estimators=50)
            xgb.fit(X_train, y_train)
            pred_xgb = xgb.predict(X_test)
            rmse_xgb = np.sqrt(mean_squared_error(y_test, pred_xgb))
        except:
            rmse_xgb = np.nan

        results.append({
            'Query': query,
            'Metric': metric,
            'ARIMA': rmse_arima,
            'Prophet': rmse_prophet,
            'LSTM': rmse_lstm,
            'RandomForest': rmse_rf,
            'XGBoost': rmse_xgb
        })

13:36:53 - cmdstanpy - INFO - Chain [1] start processing
13:36:53 - cmdstanpy - INFO - Chain [1] done processing


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step


13:36:57 - cmdstanpy - INFO - Chain [1] start processing
13:36:58 - cmdstanpy - INFO - Chain [1] done processing


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step


13:37:02 - cmdstanpy - INFO - Chain [1] start processing
13:37:02 - cmdstanpy - INFO - Chain [1] done processing


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


13:37:11 - cmdstanpy - INFO - Chain [1] start processing
13:37:12 - cmdstanpy - INFO - Chain [1] done processing


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step


13:37:16 - cmdstanpy - INFO - Chain [1] start processing
13:37:16 - cmdstanpy - INFO - Chain [1] done processing


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


## 4. Results Table

In [8]:
res_df = pd.DataFrame(results)
display(res_df.round(2))

,Query,Metric,ARIMA,Prophet,LSTM,RandomForest,XGBoost
0,Q1,CPU,10.05,1.61,2.72,2.80,2.72
1,Q1,LatencyMs,19.95,4.50,4.73,4.83,4.75
2,Q1,LogicalReads,14.00,2.03,3.49,3.85,3.79
3,Q2,CPU,10.32,1.68,3.06,2.88,3.08
4,Q2,LatencyMs,19.16,4.07,4.90,5.08,4.84
5,Q2,LogicalReads,12.43,2.14,3.46,3.33,3.35


- The table shows RMSE for each model, metric, and query (lower is better).
- Highlight the best (lowest) RMSE in each row for reporting.

## Troubleshooting LSTM NaN Values

- If you observe NaN for LSTM RMSE, this usually arises from a shape mismatch, insufficient test samples for windowing, or improper scaling/inverse-scaling.
- The above LSTM code ensures:
    - Correct handling of test windowing (includes previous lags from train).
    - Consistent scaling/inverse-scaling for both train and test set.
    - No missing values are present in LSTM input windows.
- If you still encounter NaN, check for data leakage, extreme outliers, or try increasing the test set size or epochs for training.

## 5. Interpretation
- Prophet often achieves the lowest RMSE, indicating superior handling of trend/seasonality.
- Tree-based models (RF/XGBoost) may outperform ARIMA/LSTM, but generally not Prophet.
- LSTM performance may be limited by dataset size and strong deterministic structure.